In [ ]:
pip install konlpy  #KoNLPy는한국어를 토큰화 할때 형태소 분석기를 사용하는데 
                    #그때 필요한 라이브러리 pip로 설치를 진행해준다.

     |████████████████████████████████| 19.4 MB 1.2 MB/s 
     |████████████████████████████████| 86 kB 5.0 MB/s 
     |████████████████████████████████| 448 kB 37.6 MB/s 
  Attempting uninstall: beautifulsoup4
    Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
import urllib.request
from konlpy.tag import Okt
from tqdm import tqdm
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

#훈련데이터와 테스트 데이터를 다운로드 한다.
urllib.request.urlretrieve("https://raw.githubusercontent.com/e9t/nsmc/master/ratings_train.txt", filename="ratings_train.txt")
urllib.request.urlretrieve("https://raw.githubusercontent.com/e9t/nsmc/master/ratings_test.txt", filename="ratings_test.txt")

#Pandas를 이용하여 훈련데이터와 테스트데이터를 저장한다.
train_data = pd.read_table('ratings_train.txt')
test_data = pd.read_table('ratings_test.txt')

print('훈련용 리뷰 개수 :',len(train_data))
train_data[:5]

train_data = train_data.dropna(how = 'any') # Null 값이 존재하는 행 제거
print(train_data.isnull().values.any()) # Null 값이 존재하는지 확인

print(len(train_data))

훈련용 리뷰 개수 : 150000
False
149995


In [ ]:
#현재 데이터는 영화 리뷰에 관한 데이터인데 알수없는 특수기호나 영어로 된 리뷰들은
#분석에 필요가 없기 때문에 정규식을 사용하여 제거해준다.
train_data['document'] = train_data['document'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","")# 한글과 공백을 제외하고 모두 제거
train_data['document'] = train_data['document'].str.replace('^ +', "") # white space 데이터를 empty value로 변경
train_data['document'].replace('', np.nan, inplace=True)# 공백은 Null 값으로 변경
train_data = train_data.dropna(how = 'any') # Null 값 제거


test_data['document'] = test_data['document'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","") # 정규 표현식 수행
test_data['document'] = test_data['document'].str.replace('^ +', "") # 공백은 empty 값으로 변경
test_data['document'].replace('', np.nan, inplace=True) # 공백은 Null 값으로 변경
test_data = test_data.dropna(how='any') # Null 값 제거

In [ ]:
#이제 단어를 토큰화 하는 과정이 필요하다. NLP로 분석을 진행할 때 단어를 나눠서 단어의
#의미와 문맥을 통해 분석하게 되는데 여기서 조사나 접속사 같은 불용어들은 필요가 없다
stopwords = ['의','가','이','은','들','는','좀','잘','걍','과','도','를','으로','자','에','와','한','하다']

#토큰화를 위한 형태소 분석기는 KoNLPy의 Okt를 사용한다.
okt = Okt()

X_train = []
for sentence in tqdm(train_data['document']):
    tokenized_sentence = okt.morphs(sentence, stem=True) # 토큰화
    stopwords_removed_sentence = [word for word in tokenized_sentence if not word in stopwords] # 불용어 제거
    X_train.append(stopwords_removed_sentence)


print(X_train[:3])

X_test = []
for sentence in tqdm(test_data['document']):
    tokenized_sentence = okt.morphs(sentence, stem=True) # 토큰화
    stopwords_removed_sentence = [word for word in tokenized_sentence if not word in stopwords] # 불용어 제거
    X_test.append(stopwords_removed_sentence)

100%|██████████| 148740/148740 [10:55<00:00, 226.95it/s]


[['아', '더빙', '진짜', '짜증나다', '목소리'], ['흠', '포스터', '보고', '초딩', '영화', '줄', '오버', '연기', '조차', '가볍다', '않다'], ['너', '무재', '밓었', '다그', '래서', '보다', '추천', '다']]


100%|██████████| 49575/49575 [04:19<00:00, 190.79it/s]


In [ ]:
#인공지능이 텍스트를 숫자로 처리할 수 있도록 훈련 데이터와 테스트 데이터에 정수 인코딩을
#진행해야 한다.
#단어 집합이 생성되는 동시에 각 단어에 대한 고유한 정수가 부여된다.
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train)



threshold = 3 # 등장 빈도수
total_cnt = len(tokenizer.word_index) # 단어의 수
rare_cnt = 0 # 등장 빈도수가 threshold보다 작은 단어의 개수를 카운트
total_freq = 0 # 훈련 데이터의 전체 단어 빈도수 총 합
rare_freq = 0 # 등장 빈도수가 threshold보다 작은 단어의 등장 빈도수의 총 합

# 단어와 빈도수의 쌍(pair)을 key와 value로 받는다.
for key, value in tokenizer.word_counts.items():
    total_freq = total_freq + value

    # 단어의 등장 빈도수가 threshold보다 작으면
    if(value < threshold):
        rare_cnt = rare_cnt + 1
        rare_freq = rare_freq + value

print('단어 집합(vocabulary)의 크기 :',total_cnt)
print('등장 빈도가 %s번 이하인 희귀 단어의 수: %s'%(threshold - 1, rare_cnt))
print("단어 집합에서 희귀 단어의 비율:", (rare_cnt / total_cnt)*100)
print("전체 등장 빈도에서 희귀 단어 등장 빈도 비율:", (rare_freq / total_freq)*100)

vocab_size = total_cnt - rare_cnt + 1
print('단어 집합의 크기 :',vocab_size)


tokenizer = Tokenizer(vocab_size) 
tokenizer.fit_on_texts(X_train)
X_train = tokenizer.texts_to_sequences(X_train)
X_test = tokenizer.texts_to_sequences(X_test)

y_train = np.array(train_data['label'])
y_test = np.array(test_data['label'])

단어 집합(vocabulary)의 크기 : 43752
등장 빈도가 2번 이하인 희귀 단어의 수: 24328
단어 집합에서 희귀 단어의 비율: 55.604315231303715
전체 등장 빈도에서 희귀 단어 등장 빈도 비율: 1.8637396855052155
단어 집합의 크기 : 19425


In [ ]:
#빈 샘플 제거
drop_train = [index for index, sentence in enumerate(X_train) if len(sentence) < 1]

X_train = np.delete(X_train, drop_train, axis=0)
y_train = np.delete(y_train, drop_train, axis=0)
print(len(X_train))
print(len(y_train))

#샘플들의 길이를 30으로 맞춰준다.
def below_threshold_len(max_len, nested_list):
  count = 0
  for sentence in nested_list:
    if(len(sentence) <= max_len):
        count = count + 1
  print('전체 샘플 중 길이가 %s 이하인 샘플의 비율: %s'%(max_len, (count / len(nested_list))*100))

max_len = 30
below_threshold_len(max_len, X_train)

X_train = pad_sequences(X_train, maxlen = max_len)
X_test = pad_sequences(X_test, maxlen = max_len)

/usr/local/lib/python3.7/dist-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)


148509
148509
전체 샘플 중 길이가 30 이하인 샘플의 비율: 94.44612784410373


In [ ]:
from tensorflow.keras.layers import Embedding, Dense, LSTM              #단어 임베딩, 시계열 데이터 처리를 위한 LSTM
from tensorflow.keras.models import Sequential                          #MLP층을 쌓아가기 위함
from tensorflow.keras.models import load_model                          #학습된 모델 저장
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint   #조기멈춤, 개선된 모델 저장

#임베딩 벡터의 차원은 100
embedding_dim = 100
hidden_units = 128

#단어 문맥의 파악이 필요하므로 LSTM을 사용하여 학습한다.
model = Sequential()
model.add(Embedding(vocab_size, embedding_dim))
model.add(LSTM(hidden_units))
model.add(Dense(1, activation='sigmoid'))

#정확도가 개선되지 않을경우 조기 종료를 진행, 정확도가 개선되는 경우에만 모델을 저장
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=4)
mc = ModelCheckpoint('best_model.h5', monitor='val_acc', mode='max', verbose=1, save_best_only=True)

#모델 실행, 15번 반복
model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc'])
history = model.fit(X_train, y_train, epochs=15, callbacks=[es, mc], batch_size=64, validation_split=0.2)

#저장된 모델의 정확도를 측정
loaded_model = load_model('best_model.h5')
print("\n 테스트 정확도: %.4f" % (loaded_model.evaluate(X_test, y_test)[1]))

Epoch 1/15
1857/1857 [==============================] - ETA: 0s - loss: 0.3860 - acc: 0.8250
Epoch 00001: val_acc improved from -inf to 0.84762, saving model to best_model.h5
1857/1857 [==============================] - 174s 91ms/step - loss: 0.3860 - acc: 0.8250 - val_loss: 0.3520 - val_acc: 0.8476
Epoch 2/15
1857/1857 [==============================] - ETA: 0s - loss: 0.3251 - acc: 0.8590
Epoch 00002: val_acc improved from 0.84762 to 0.85489, saving model to best_model.h5
1857/1857 [==============================] - 168s 90ms/step - loss: 0.3251 - acc: 0.8590 - val_loss: 0.3417 - val_acc: 0.8549
Epoch 3/15
1857/1857 [==============================] - ETA: 0s - loss: 0.3005 - acc: 0.8729
Epoch 00003: val_acc improved from 0.85489 to 0.85913, saving model to best_model.h5
1857/1857 [==============================] - 167s 90ms/step - loss: 0.3005 - acc: 0.8729 - val_loss: 0.3285 - val_acc: 0.8591
Epoch 4/15
1857/1857 [==============================] - ETA: 0s - loss: 0.2810 - acc: 0.883

In [ ]:
#학습된 결과를 가지고 리뷰를 예측해본다. 기존에 학습할때 가공했던것처럼 입력데이터를 가공 후 예측
def sentiment_predict(new_sentence):
  new_sentence = re.sub(r'[^ㄱ-ㅎㅏ-ㅣ가-힣 ]','', new_sentence)
  new_sentence = okt.morphs(new_sentence, stem=True) # 토큰화
  new_sentence = [word for word in new_sentence if not word in stopwords] # 불용어 제거
  encoded = tokenizer.texts_to_sequences([new_sentence]) # 정수 인코딩
  pad_new = pad_sequences(encoded, maxlen = max_len) # 패딩
  score = float(loaded_model.predict(pad_new)) # 예측
  if(score > 0.5):
    print("{:.2f}% 확률로 긍정 리뷰입니다.\n".format(score * 100))
  else:
    print("{:.2f}% 확률로 부정 리뷰입니다.\n".format((1 - score) * 100))

sentiment_predict('이 영화 개꿀잼 ㅋㅋㅋ')
sentiment_predict('이 영화 핵노잼 ㅠㅠ')

90.24% 확률로 긍정 리뷰입니다.

97.71% 확률로 부정 리뷰입니다.

